In [24]:
from google.colab import drive
drive.mount("/content/drive")
!pip install biopython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from Bio import Entrez, SeqIO
import pandas , csv
CSVs = ["Ino_Prot.csv","Myo_Prot.csv","Podo_Prot.csv","Sipho_Prot.csv"]
SeqCSVs = ["Ino_Prot_Seq.csv","Myo_Prot_Seq.csv","Podo_Prot_Seq.csv","Sipho_Prot_Seq.csv"]
dataL = []
def readCSV():
  for csv in CSVs:
    data = pandas.read_csv("/content/drive/My Drive/FYP/"+csv)
    dataL.append(data)
def getProtSeqByID(id:str):
  Entrez.email = "cyli77-c@my.cityu.edu.hk"  
  handle = Entrez.esearch(db="protein", term=id)
  read_gene = Entrez.read(handle)
  #print(read_gene['IdList'])
  handle = Entrez.efetch(db="protein", id=read_gene['IdList'], rettype="fasta", retmode="text")
  #print (handle.read())
  seq_record = SeqIO.read(handle, "fasta")
  print (seq_record.seq)
def writeSeq2CSV():
  for seqcsv in SeqCSVs:
    with open(seqcsv,"w",newline="") as csvfile:
      writer = csv.writer(csvfile)
      for data in dataL:
        for i in range(len(data)):
          protTag = data["prot_tag"][i]
          protId = data["prot_id"][i]
          protName = data["name"][i]
          protLength = data["size"][i]
          print(str(protId))
          getProtSeqByID(str(protId))
          break
readCSV()
writeSeq2CSV()

AAL40840.1
MFGYDYVSNHCSYLITTCSIYSFRFLTMIYAIVGRPRSGKSYESVVYHIIPAIQSGRKVITNIPLNIPMFEKVFGESAKYLIKVIDAQFTEYGSMNRPFSKVEHYLDDWRDGKNRAPLYVIDEAHMVIPTRLGDPKILEFYSMHGHYGIDIIILTQNLRKIHADIRAMIEMTYYCAKNTAFGSKKTYTKKVRIGDTREDINIEQRTYKEHYFGFYQSHTQSSGAVDEAQAFDITPIWKRWPFWGSLICFIIVILILGYYFQSRKSKTIDAQPLPSEQVEQVPLSVPPQSPSSKPQQTIKTVPVSEPLKDFNLYVSGHAKQIAYKKMSFSREIDTKLTFYHVYISAYQDDQFSFSLNNIDLEKMGYQFEALTECVYRITWGSNSCVITCIDESRFNQQKAETVFDHVPKLGI
NP_899486.1
MDSHRAEIIRLKRRVSKLEELVKTLMNRDVTPRNVFDEEDEHISLHPVDDTFDMFRIKRELLQAMSQQSVRYLDITFENYPDVARHKVYQNHLMQDVGMSDADRKKLAKTRYMIADEMSVGDEFECPGCGEKVIKKSYQHRFCGVKKQGQSNCKDFINNWFNPKRLERTLEWLEK
ASU01169.1
MALISQSIKNLKGGISQQPDILRYSDQGSKQINGFSSEVEGLQKRPPSVHVKRLTDQFGLGQKPYCHIINRDEVERYAVFFTGSNIRVFDLFTGDEKTVNAPNGLSYVTSSNPRKDLRMVTVADYTFILNRNVSTAQGTTNTPSGLAPFGHFGLVVIRGGQYGRTYRVKVNGSVEASFETPLGDRVEHAKQIDIAYIIDQLAAGLINRGWAVTKGSGYFYFSKSGTVIINSLEVEDGYNGQLAWGIINDVQKTTQLPVYAPNNYIIRVSGDPTLNQDDYYVKFDASRNVWTECPAPNIKADYNKDTMPHVLIREADGTFTFKQADWTHRAAGDDDTNPYPSFIGNSINDIFFFRNRLGFLSGENVILSGSGNYFNFFP

KeyboardInterrupt: ignored

In [ ]:
!pip install biopython
from Bio import Entrez
hd_info = Entrez.einfo()
# get the database onfo
read_info = Entrez.read(hd_info)
"""
for db in read_info['DbList']:
    print (db)
"""

ModuleNotFoundError: ignored

# Download Data from NCBI


In [36]:
from Bio import Entrez, SeqIO
from datetime import datetime
import time ,random
from multiprocessing import Process
from multiprocessing import Pool
import multiprocessing
import pandas, csv

# CSVs = ["Ino_Prot.csv","Myo_Prot.csv","Podo_Prot.csv","Sipho_Prot.csv"]
# SeqCSVs = ["Ino_Prot_Seq.csv","Myo_Prot_Seq.csv","Podo_Prot_Seq.csv","Sipho_Prot_Seq.csv"]
CSVs = ["Ino_Prot.csv"]
SeqCSVs = ["Ino_Prot_Seq.csv"]
dataL = []


def readCSV():
    for csv in CSVs:
        try:
            data = pandas.read_csv("/content/drive/My Drive/FYP/"+csv)
            data = data.dropna(how='all')
            print(data)
        except Exception as e:
            print(e)
            data = data.dropna(how='all')
            data = pandas.read_csv(csv)
            print(data)
        dataL.append(data)


def getProtSeqByID(id: str):
    count = 0
    while count < 4:
        try:
            time.sleep(random.randrange(8))
            print("{} start fetching".format(id))
            Entrez.email = "cyli7-c@my.cityu.edu.hk"
            handle = Entrez.esearch(db="protein", term=id)
            read_gene = Entrez.read(handle)
            # print(read_gene['IdList'])
            handle = Entrez.efetch(
                db="protein", id=read_gene["IdList"], rettype="fasta", retmode="text"
            )
            # print (handle.read())
            seq_record = SeqIO.read(handle, "fasta")
            # print (seq_record.seq)
            print("{} sucessfully fetched !".format(id))
            return (seq_record.seq , seq_record.description)
        except Exception as e:
            # Try agin if any error
            count += 1
            print("{} is trying to fetch again".format(id))
            print(e)
    return ("","")


def writeSeq2CSV():
    for seqcsv, data in zip(SeqCSVs, dataL):
        with open(seqcsv, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(
                ["prot_tag", "prot_id", "name", "size", "seq", "description"]
            )
            finishedCount = 0
            # Start loop
            PROCESS_NUM = 5
            while finishedCount != len(data):
                remainCount = len(data) - finishedCount
                processCount = 0
                # For create process
                if remainCount > PROCESS_NUM:
                    processCount = PROCESS_NUM
                else:
                    processCount = remainCount
                # each time processCount
                
                protTags = []
                protIds = []
                protNames = []
                protLengths = []
                processL = []
                
                for i in range(processCount):
                    protTags.append(data["prot_tag"][finishedCount + i])
                    protIds.append(data["prot_id"][finishedCount + i])
                    protNames.append(data["name"][finishedCount + i])
                    protLengths.append(data["size"][finishedCount + i])
                    now = datetime.now()
                    curTime = now.strftime("%H:%M:%S")
                    print(
                        "{} {} Doing index:{} {}".format(curTime, seqcsv,finishedCount + i , protIds[i])
                    )
                finishedCount += processCount
                pool = Pool(processCount)
                pool_outputs = pool.map(getProtSeqByID, protIds)
                pool.close()
                pool.join() 
                for i in range(processCount):
                    seq = pool_outputs[i][0]
                    des = pool_outputs[i][1]
                    writer.writerow(
                        [
                            protTags[i],
                            protIds[i],
                            protNames[i],
                            protLengths[i],
                            seq,
                            des,
                        ]
                    )
            csvfile.truncate()
            csvfile.close()
            print("Finished CSV {}".format(seqcsv))

readCSV()
writeSeq2CSV()

       prot_tag         prot_id  ... cluster* origin*
0    AAL40840.1      AAL40840.1  ...     10-3    10-3
1   VCYphi_gp08  YP_004934230.1  ...     10-3    10-3
2   K05K4_52840      ARP21986.1  ...     10-3    10-3
3    AV950_gp11  YP_004286239.1  ...     10-3    10-3
4   K05K4_52730      ARP21975.1  ...     10-3    10-3
..          ...             ...  ...      ...     ...
93  K05K4_52940      ARP21994.1  ...        6       6
94       fs1p01     NP_695190.1  ...        6       6
95    VGJphip04     NP_835475.1  ...        6       6
96   BAJ12070.1      BAJ12070.1  ...        6       6
97   BAA33516.1      BAA33516.1  ...        6       6

[98 rows x 10 columns]
05:40:29 Ino_Prot_Seq.csv Doing index:0 AAL40840.1
05:40:29 Ino_Prot_Seq.csv Doing index:1 YP_004934230.1
05:40:29 Ino_Prot_Seq.csv Doing index:2 ARP21986.1
05:40:29 Ino_Prot_Seq.csv Doing index:3 YP_004286239.1
05:40:29 Ino_Prot_Seq.csv Doing index:4 ARP21975.1
AAL40840.1 start fetching
AAL40840.1 sucessfully fetched !
YP_004

Sliding Window


In [ ]:
import numpy as np
data = np.arange(20)
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
